# 93. CNN vs MLP：帰納バイアスの効果を実証

## 学習目標

このノートブックでは、以下を学びます：

1. **MLP（多層パーセプトロン）**の構造と特性
2. **CNN vs MLP**のパラメータ効率比較
3. **実験による性能比較**
4. **帰納バイアスの効果**の実証

## 目次

1. [MLPの復習](#section1)
2. [構造の比較](#section2)
3. [MNIST実験](#section3)
4. [位置シフト実験](#section4)
5. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

<a id="section1"></a>
## 1. MLPの復習

### MLP（多層パーセプトロン）とは

- 全結合層のみで構成されるニューラルネットワーク
- 各ニューロンは前層の**すべての**ニューロンと接続
- 空間的な構造を考慮しない（画像を1次元ベクトルとして扱う）

In [ ]:
def visualize_mlp_vs_cnn_structure():
    """MLPとCNNの構造比較"""
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # MLP
    ax = axes[0]
    
    # 入力層（flatten済み）
    input_neurons = 6
    for i in range(input_neurons):
        circle = plt.Circle((0.2, 0.9 - i*0.12), 0.03, color='lightblue', ec='blue')
        ax.add_patch(circle)
    ax.text(0.2, 0.97, '入力\n(flatten)', ha='center', fontsize=10)
    
    # 隠れ層
    hidden_neurons = 4
    for i in range(hidden_neurons):
        circle = plt.Circle((0.5, 0.8 - i*0.15), 0.03, color='lightyellow', ec='orange')
        ax.add_patch(circle)
    ax.text(0.5, 0.92, '隠れ層', ha='center', fontsize=10)
    
    # 出力層
    output_neurons = 3
    for i in range(output_neurons):
        circle = plt.Circle((0.8, 0.75 - i*0.15), 0.03, color='lightgreen', ec='green')
        ax.add_patch(circle)
    ax.text(0.8, 0.92, '出力', ha='center', fontsize=10)
    
    # 全結合の接続（一部）
    for i in range(input_neurons):
        for j in range(hidden_neurons):
            ax.plot([0.23, 0.47], [0.9 - i*0.12, 0.8 - j*0.15], 
                   'gray', alpha=0.2, linewidth=0.5)
    
    for i in range(hidden_neurons):
        for j in range(output_neurons):
            ax.plot([0.53, 0.77], [0.8 - i*0.15, 0.75 - j*0.15], 
                   'gray', alpha=0.2, linewidth=0.5)
    
    ax.text(0.5, 0.1, '全結合：すべてのニューロンが接続\n→ 空間構造を無視', 
           ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('MLP（多層パーセプトロン）', fontsize=14, fontweight='bold')
    
    # CNN
    ax = axes[1]
    
    # 入力（2Dグリッド）
    for i in range(3):
        for j in range(3):
            rect = plt.Rectangle((0.1 + j*0.06, 0.7 - i*0.06), 0.05, 0.05,
                                 facecolor='lightblue', edgecolor='blue')
            ax.add_patch(rect)
    ax.text(0.19, 0.82, '入力\n(2D)', ha='center', fontsize=10)
    
    # カーネル
    for i in range(2):
        for j in range(2):
            rect = plt.Rectangle((0.38 + j*0.04, 0.62 - i*0.04), 0.03, 0.03,
                                 facecolor='lightyellow', edgecolor='orange', linewidth=2)
            ax.add_patch(rect)
    ax.text(0.42, 0.72, 'カーネル\n(共有)', ha='center', fontsize=10)
    
    # 出力（特徴マップ）
    for i in range(2):
        for j in range(2):
            rect = plt.Rectangle((0.65 + j*0.06, 0.65 - i*0.06), 0.05, 0.05,
                                 facecolor='lightgreen', edgecolor='green')
            ax.add_patch(rect)
    ax.text(0.71, 0.82, '特徴マップ\n(2D)', ha='center', fontsize=10)
    
    # 局所接続を示す
    ax.annotate('', xy=(0.35, 0.6), xytext=(0.25, 0.6),
               arrowprops=dict(arrowstyle='->', color='red', lw=2))
    ax.annotate('', xy=(0.62, 0.6), xytext=(0.52, 0.6),
               arrowprops=dict(arrowstyle='->', color='red', lw=2))
    
    ax.text(0.5, 0.1, '局所接続 + 重み共有\n→ 空間構造を活用', 
           ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='lightgreen'))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('CNN（畳み込みニューラルネットワーク）', fontsize=14, fontweight='bold')
    
    plt.suptitle('MLP vs CNN の構造比較', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_mlp_vs_cnn_structure()

<a id="section2"></a>
## 2. 構造の比較

### モデル定義

In [ ]:
class MLP(nn.Module):
    """シンプルなMLP"""
    def __init__(self, input_size=784, hidden_size=256, num_classes=10):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class SimpleCNN(nn.Module):
    """シンプルなCNN"""
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # 28->14
        x = self.pool(self.relu(self.conv2(x)))  # 14->7
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# パラメータ数を比較
mlp = MLP()
cnn = SimpleCNN()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

print("パラメータ数の比較")
print("="*50)
print(f"MLP:  {count_parameters(mlp):,}")
print(f"CNN:  {count_parameters(cnn):,}")
print(f"比率: MLP/CNN = {count_parameters(mlp)/count_parameters(cnn):.2f}倍")

In [ ]:
def analyze_model_layers():
    """各層のパラメータ数を分析"""
    print("\n" + "="*60)
    print("層ごとのパラメータ数")
    print("="*60)
    
    print("\n【MLP】")
    for name, param in mlp.named_parameters():
        print(f"  {name}: {param.shape} = {param.numel():,}")
    
    print("\n【CNN】")
    for name, param in cnn.named_parameters():
        print(f"  {name}: {param.shape} = {param.numel():,}")

analyze_model_layers()

<a id="section3"></a>
## 3. MNIST実験

MNISTデータセットでMLPとCNNを比較します。

In [ ]:
# データの準備
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# MNISTをダウンロード
try:
    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('./data', train=False, transform=transform)
    
    # 小さいサブセットで実験（高速化のため）
    train_subset = torch.utils.data.Subset(train_dataset, range(5000))
    test_subset = torch.utils.data.Subset(test_dataset, range(1000))
    
    train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)
    
    print(f"訓練データ: {len(train_subset)}")
    print(f"テストデータ: {len(test_subset)}")
except Exception as e:
    print(f"データのダウンロードに失敗: {e}")
    print("合成データで代用します")
    
    # 合成データ
    X_train = torch.randn(5000, 1, 28, 28)
    y_train = torch.randint(0, 10, (5000,))
    X_test = torch.randn(1000, 1, 28, 28)
    y_test = torch.randint(0, 10, (1000,))
    
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)

In [ ]:
def train_model(model, train_loader, test_loader, epochs=5, lr=0.001):
    """モデルを訓練"""
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    train_losses = []
    test_accs = []
    
    for epoch in range(epochs):
        # 訓練
        model.train()
        total_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        train_losses.append(total_loss / len(train_loader))
        
        # テスト
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        test_acc = 100 * correct / total
        test_accs.append(test_acc)
        
        print(f"Epoch {epoch+1}/{epochs}: Loss={train_losses[-1]:.4f}, Acc={test_acc:.2f}%")
    
    return train_losses, test_accs

# MLPの訓練
print("\n" + "="*50)
print("MLPの訓練")
print("="*50)
mlp = MLP().to(device)
mlp_losses, mlp_accs = train_model(mlp, train_loader, test_loader, epochs=10)

# CNNの訓練
print("\n" + "="*50)
print("CNNの訓練")
print("="*50)
cnn = SimpleCNN().to(device)
cnn_losses, cnn_accs = train_model(cnn, train_loader, test_loader, epochs=10)

In [ ]:
def plot_training_comparison():
    """訓練結果の比較"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 損失
    axes[0].plot(mlp_losses, 'b-o', label='MLP')
    axes[0].plot(cnn_losses, 'r-s', label='CNN')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('訓練損失')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 精度
    axes[1].plot(mlp_accs, 'b-o', label='MLP')
    axes[1].plot(cnn_accs, 'r-s', label='CNN')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('テスト精度')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle('MLP vs CNN の学習曲線', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"\n最終テスト精度:")
    print(f"  MLP: {mlp_accs[-1]:.2f}%")
    print(f"  CNN: {cnn_accs[-1]:.2f}%")

plot_training_comparison()

<a id="section4"></a>
## 4. 位置シフト実験

平行移動に対する頑健性を比較します。

In [ ]:
def test_shift_robustness(model, test_loader, shift_amounts):
    """シフトに対する頑健性をテスト"""
    model.eval()
    results = []
    
    for shift in shift_amounts:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                # 画像をシフト
                if shift != 0:
                    data = torch.roll(data, shifts=shift, dims=2)  # 水平シフト
                
                data, target = data.to(device), target.to(device)
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        acc = 100 * correct / total
        results.append(acc)
    
    return results

# シフト量のリスト
shift_amounts = list(range(-10, 11, 2))

# テスト
mlp_shift_accs = test_shift_robustness(mlp, test_loader, shift_amounts)
cnn_shift_accs = test_shift_robustness(cnn, test_loader, shift_amounts)

# 結果をプロット
plt.figure(figsize=(12, 6))
plt.plot(shift_amounts, mlp_shift_accs, 'b-o', label='MLP', linewidth=2, markersize=8)
plt.plot(shift_amounts, cnn_shift_accs, 'r-s', label='CNN', linewidth=2, markersize=8)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('水平シフト量（ピクセル）', fontsize=12)
plt.ylabel('精度 (%)', fontsize=12)
plt.title('位置シフトに対する頑健性\nCNNは重み共有により位置変化に強い', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nシフトに対する精度低下:")
print(f"  MLP: シフト0→±10で {mlp_shift_accs[5]:.1f}% → {mlp_shift_accs[0]:.1f}%")
print(f"  CNN: シフト0→±10で {cnn_shift_accs[5]:.1f}% → {cnn_shift_accs[0]:.1f}%")

<a id="summary"></a>
## 5. まとめ

### 実験結果

| 観点 | MLP | CNN |
|------|-----|-----|
| パラメータ数 | 多い | 少ない |
| 学習速度 | 遅い | 速い |
| 最終精度 | 低い | 高い |
| シフト頑健性 | 低い | 高い |

### 帰納バイアスの効果

CNNの帰納バイアス（局所性、重み共有、平行移動等変性）は：
- パラメータ効率を向上
- 学習を高速化
- 汎化性能を向上
- 位置変化への頑健性を付与

### 次のノートブック

次のノートブックでは、**CNNが苦手なケース**について学びます。